In [1]:
import torch
from torch import nn
from torch.nn import functional as F

自定义块

In [4]:
class MLP(nn.Module):
    # 模型参数声明层（包含两个全连接层）
    def __init__(self):
        # 调用MLP的父类Module的构造函数执行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # 模型的前向传播 forward
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [5]:
X = torch.rand(2, 20)
net = MLP()
net(X)

tensor([[ 0.2718, -0.0396, -0.0647, -0.2325,  0.0419,  0.0240, -0.1596, -0.1251,
          0.1388, -0.0075],
        [ 0.0376, -0.0958, -0.0943, -0.2466,  0.0337, -0.1677, -0.0272, -0.1369,
          0.0733,  0.0139]], grad_fn=<AddmmBackward0>)

顺序块

In [11]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # args: 所有层的列表
        for idx, module in enumerate(args):
            # idx：0，1，...
            # module：块
            # module是Module的子类实例
            # _modules 类型是 OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [12]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.2137,  0.0501, -0.2648, -0.0127, -0.1741,  0.1579,  0.1512, -0.0585,
         -0.0380,  0.0884],
        [ 0.2740,  0.1798, -0.2194,  0.0666, -0.2904,  0.2694,  0.0684, -0.0078,
          0.0750,  0.0329]], grad_fn=<AddmmBackward0>)

常数参数和集合其他代码

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # requires_grad=False：不计算梯度，训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用 self.linear：两个全连接层共享参数
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
net = FixedHiddenMLP()
net(X)

tensor(0.1265, grad_fn=<SumBackward0>)

In [16]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [17]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3134, grad_fn=<SumBackward0>)